In [1]:
from pylsl import StreamInfo, StreamOutlet
import numpy as np
import pandas as pd
import time
from psychopy import visual, core, event
from glob import glob
from random import choice, random
from psychopy.visual import ShapeStim
from pylsl import StreamInlet, resolve_byprop

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#define a function to get frame on and off

#Assuming that you use a 60 Hz monitor:

#8.57 Hz corresponds to 60/8.57 = 7 frames on and 7 frames off.
#10 Hz --> 6 frames
#12 Hz --> 5 frames
#15 Hz --> 4 frames

#here assuming that the frequency is frequency of SHIFTS (8.57 shifts per seconds) 
#rather than CYCLES (8.57 on and offs per second) since the latter would be 
#impossible on a 60 Hz monitor, because you should then change the image between frame 3 and 4.

import math
def getFrames(freq):
    framerate = 60 # mywin.getActualFrameRate()
    frame = int(round(framerate / freq))
    frame_on = math.ceil(frame / 2)
    frame_off = math.floor(frame / 2)
    return frame_on, frame_off

In [3]:
#Author: Apiporn Simapornchai
def three_stimuli_blinking(frame_on1, frame_off1, frame_on2, frame_off2, frame_on3, frame_off3, shapes, flipCount):
    looptime = math.gcd(frame_on1,math.gcd(frame_on2,frame_on3))
    print("loop time: ",looptime)
#   print(frame_on1, frame_off1, frame_on2, frame_off2, frame_on3, frame_off3)
#   print(flipCount)
    
    #reset clock for next trial
    trialclock.reset()
    while trialclock.getTime()<soa:
        if(flipCount == 0 or (flipCount%frame_on1 ==0 and flipCount%(frame_on1*2) !=0)):
            shapes[0].setAutoDraw(True)
            shapes[1].setAutoDraw(False)
        if(flipCount%(frame_off1*2) ==0):
            shapes[1].setAutoDraw(True)
            shapes[0].setAutoDraw(False)

        if(flipCount == 0 or(flipCount%frame_on2 ==0 and flipCount%(frame_on2*2) !=0)):
            shapes[2].setAutoDraw(True)
            shapes[3].setAutoDraw(False)
        if(flipCount%(frame_off2*2) ==0):
            shapes[3].setAutoDraw(True)
            shapes[2].setAutoDraw(False)

        if(flipCount == 0 or(flipCount%frame_on3 ==0 and flipCount%(frame_on3*2) !=0)):
            shapes[4].setAutoDraw(True)
            shapes[5].setAutoDraw(False)
        if(flipCount%(frame_off3*2) ==0):
            shapes[5].setAutoDraw(True)
            shapes[4].setAutoDraw(False)

        for frameN in range(looptime):
            mywin.flip()
            flipCount+=1
            print(trialclock.getTime())
            

In [4]:
#define a marker stream to receive the predicted class

marker_streams = resolve_byprop('type', 'Markers', timeout=3)
if marker_streams:
    inlet_marker = StreamInlet(marker_streams[0])
    marker_time_correction = inlet_marker.time_correction()
    print("Found Markers stream")
else:
    inlet_marker = False
    print("Cant find Markers stream")

markers = []
def marker_result():
    marker, timestamp = inlet_marker.pull_sample(timeout=3)
    return marker
        

Found Markers stream


In [5]:
#Author: Apiporn Simapornchai
def stimuli_blinking_nonstop(frame_on1, frame_off1, frame_on2, frame_off2, frame_on3, frame_off3, shapes, flipCount):
    looptime = math.gcd(frame_on1,math.gcd(frame_on2,frame_on3))

    arrow =ShapeStim(mywin, vertices=arrowVert, fillColor='darkred', size=.5, lineColor='red', pos=arrowSequence[0])
    result = marker_result()
    
    while(True):
        if flipCount ==60:
            flipCount = 0
        if(flipCount == 0 or (flipCount%frame_on1 ==0 and flipCount%(frame_on1*2) !=0)):
            shapes[0].setAutoDraw(True)
            shapes[1].setAutoDraw(False)
        if(flipCount%(frame_off1*2) ==0):
            shapes[1].setAutoDraw(True)
            shapes[0].setAutoDraw(False)

        if(flipCount == 0 or(flipCount%frame_on2 ==0 and flipCount%(frame_on2*2) !=0)):
            shapes[2].setAutoDraw(True)
            shapes[3].setAutoDraw(False)
        if(flipCount%(frame_off2*2) ==0):
            shapes[3].setAutoDraw(True)
            shapes[2].setAutoDraw(False)

        if(flipCount == 0 or(flipCount%frame_on3 ==0 and flipCount%(frame_on3*2) !=0)):
            shapes[4].setAutoDraw(True)
            shapes[5].setAutoDraw(False)
        if(flipCount%(frame_off3*2) ==0):
            shapes[5].setAutoDraw(True)
            shapes[4].setAutoDraw(False)
        
        time = trialclock.getTime()
        
        #if the predicted class is received from online_classification.py, print the corresponding arrow
        if (result):
            arrow.setAutoDraw(False)
            arrow = ShapeStim(mywin, vertices=arrowVert, fillColor='darkred', size=.5, lineColor='red', pos=arrowSequence[result-1])
            arrow.setAutoDraw(True)           
            
        elif result==len(arrowSequence) and time>soa:
            arrow.setAutoDraw(False)
            
        for frameN in range(looptime):
            mywin.flip()
            flipCount+=1
            

In [6]:
#setting params
mywin = visual.Window([1920, 1080], fullscr=False)

soa = 2  #stimulus onset asynchrony
iti = 1  #inter trial interval

test_freq = [6, 10, 15]  #, 15]
stimuli_seq = [0,1,2] * 2  #five trials for each freq in test_freq
freq_len = len(test_freq)

frame_on1, frame_off1 = getFrames(test_freq[0])
frame_on2, frame_off2 = getFrames(test_freq[1])
frame_on3, frame_off3 = getFrames(test_freq[2])

count = 0
trialclock = core.Clock()

frame_on = 0
frame_off = 0

patternup1Pos = [0, 0.5]
patternright1Pos = [0.5, 0]
patternleft1Pos =[-0.5, 0]

# Arrow position is now y+0.2
arrowUp1Pos = [0, 0.7]
arrowRigh1Pos = [0.5, 0.2]
arrowLeft1Pos=[-0.5, 0.2]

# array to identify the sequenct of the stimuli
arrowSequence = [arrowUp1Pos,arrowRigh1Pos,arrowLeft1Pos]

patternup1 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern1', autoLog=False, color=[1,1,1], pos=patternup1Pos)
patternup2 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern2', autoLog=False, color=[-1,-1,-1], pos=patternup1Pos)

patternright1 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern1', autoLog=False, color=[1,1,1], pos=patternright1Pos)
patternright2 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern2', autoLog=False, color=[-1,-1,-1], pos=patternright1Pos)

#patterndown1 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
#    name='pattern1', autoLog=False, color=[1,1,1], pos=(0, -0.5))
#patterndown2 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
#    name='pattern2', autoLog=False, color=[-1,-1,-1], pos=(0, -0.5))

patternleft1 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern1', autoLog=False, color=[1,1,1], pos=patternleft1Pos)
patternleft2 = visual.GratingStim(mywin, tex=None, sf=0, size=0.3,
    name='pattern2', autoLog=False, color=[-1,-1,-1], pos=patternleft1Pos)


# prepare the arrow shape
arrowVert = [(0,0),(-0.1,0.15),(-0.05,0.15),(-0.05,0.3),(0.05,0.3),(0.05,0.15),(0.1,0.15)]

shapes = [patternup1, patternup2, patternright1, patternright2, patternleft1, patternleft2]

#fixation cross
fixation = visual.ShapeStim(mywin, 
    vertices=((0, -0.5), (0, 0.5), (0,0), (-0.5,0), (0.5, 0)),
    lineWidth=5,
    closeShape=False,
    lineColor="white"
)


In [ ]:
#running the actual experiment
while True:

    message = visual.TextStim(mywin, text='Start recording and press space to continue')
    message.draw()
    mywin.flip()
    keys = event.getKeys()
    
    if 'space' in keys:  # If space has been pushed
        message.setText = ''
        message.draw()
        mywin.flip()  
        
        fixation.draw()
        mywin.flip() #refresh
        core.wait(iti)
        mywin.flip()
        flipCount=0
        
        stimuli_blinking_nonstop(frame_on1, frame_off1, frame_on2, frame_off2, frame_on3, frame_off3, shapes, flipCount)
            
mywin.close()  #do not delete, otherwise, the window will not turn off